In [1]:
##################################################################
#           《Python人工智能编程实践（2024年度版）》开源代码
#-----------------------------------------------------------------
#            @章节号：6.4.2（残差神经网络的TensorFlow实践）                                          
#            @作者：范淼、徐晟桐 
#            @购书链接：暂无
#            @电子邮箱：fm12@tsinghua.org.cn             
#            @官方交流QQ群号：561500762                        
##################################################################

In [2]:
from tensorflow.keras import models, layers, losses, optimizers, Model
from tensorflow import nn


#设置超参数。
FILTERS = (32, 64)
HIDDEN_SIZE = 256
NUM_CLASSES = 10
EPOCHS = 5
BATCH_SIZE = 64
LEARNING_RATE = 1e-3


class ResNet(Model):
    '''
    自定义残差神经网络类，继承自keras.Model。
    '''
    def __init__(self, filters, hidden_units, num_classes):
        
        super(ResNet, self).__init__()
        
        self.conv2a = layers.Conv2D(filters[0], kernel_size=3, padding='same', activation='relu')

        self.conv2b = layers.Conv2D(filters[1], kernel_size=3, padding='same')

        self.pool = layers.MaxPooling2D(pool_size=2, strides=2)
        
        self.flat = layers.Flatten()
        
        self.l1 = layers.Dense(hidden_units, activation='relu')
        
        self.l2 = layers.Dense(num_classes)

        
    def call(self, input_tensor):
        
        out = self.conv2a(input_tensor)

        out = self.conv2b(out)

        out += input_tensor
        
        out = nn.relu(out)
        
        out = self.pool(out)
        
        out = self.flat(out)
        
        out = self.l1(out)
        
        out = self.l2(out)
         
        return out

    
#初始化残差神经网络。
model = ResNet(FILTERS, HIDDEN_SIZE, NUM_CLASSES)

#设定神经网络的损失函数、优化方式，以及评估方法。
model.compile(optimizer=optimizers.Adam(LEARNING_RATE),
              loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

2023-03-17 17:37:49.868048: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 17:37:56.020567: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import pandas as pd


#使用pandas，读取fashion_mnist的训练和测试数据文件。
train_data = pd.read_csv('../datasets/fashion_mnist/fashion_mnist_train.csv')
test_data = pd.read_csv('../datasets/fashion_mnist/fashion_mnist_test.csv')

#从训练数据中，拆解出训练特征和类别标签。
X_train = train_data[train_data.columns[1:]]
y_train = train_data['label']

#从测试数据中，拆解出测试特征和类别标签。
X_test = test_data[train_data.columns[1:]]
y_test = test_data['label']

In [4]:
from sklearn.preprocessing import StandardScaler


#初始化数据标准化处理器。
ss = StandardScaler()

#标准化训练数据特征。
X_train = ss.fit_transform(X_train)

#标准化测试数据特征。
X_test = ss.transform(X_test)

In [5]:
X_train = X_train.reshape([-1, 28, 28, 1])

#使用fashion_mnist的训练集数据训练网络模型。
model.fit(X_train, y_train.values, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1)

Epoch 1/5
938/938 [==============================] - 213s 224ms/step - loss: 0.5182 - accuracy: 0.8402
Epoch 2/5
938/938 [==============================] - 176s 187ms/step - loss: 0.2653 - accuracy: 0.9037
Epoch 3/5
938/938 [==============================] - 175s 186ms/step - loss: 0.1960 - accuracy: 0.9282
Epoch 4/5
938/938 [==============================] - 146s 156ms/step - loss: 0.1464 - accuracy: 0.9457
Epoch 5/5
938/938 [==============================] - 113s 120ms/step - loss: 0.1075 - accuracy: 0.9598


In [6]:
X_test = X_test.reshape([-1, 28, 28, 1])

#使用fashion_mnist的测试集数据评估网络模型的效果。
result = model.evaluate(X_test, y_test.values, verbose=0)

print('残差神经网络（TensorFlow版本）在fashion_mnist测试集上的准确率为: %.2f%%。' %(result[1] * 100))

残差神经网络（TensorFlow版本）在fashion_mnist测试集上的准确率为: 92.78%。
